# Example of using fault detectors

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath(".."))
from fdi_flow.detectors.sklearn_detectors import KNNFaultDetector, RandomForestFaultDetector, \
    SVMFaultDetector, GradientBoostingFaultDetector
from fdi_flow.detectors.lgbm_detectors import LGBMFaultDetector
from fdi_flow.detectors.xgb_detectors import XGBFaultDetector
from fdi_flow.detectors.catboost_detector import CatBoostFaultDetector
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## Create example data

In [8]:
# numpy data
X1 = np.random.rand(100, 10)  # 100 samples, 10 features
y1 = np.random.randint(0, 3, 100)  # 3 classes

In [9]:
# pd.Dataframe data
X2, y2 = make_classification(
    n_samples=1000,
    n_features=5,
    n_classes=3,
    n_clusters_per_class=1,
    random_state=42
)

feature_names = [f'sensor_{i+1}' for i in range(X2.shape[1])]
df = pd.DataFrame(X2, columns=feature_names)
df['fault_class'] = y2
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
X2_train = train_df.drop('fault_class', axis=1)
y2_train = train_df['fault_class']
X2_test = test_df.drop('fault_class', axis=1)
y2_test = test_df['fault_class']

df.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,fault_class
0,0.789678,0.875056,-0.524088,1.108592,-1.402916,0
1,-1.686931,3.324227,-0.190503,3.101080,-3.083091,2
2,-1.070137,2.950171,0.508269,2.853285,-2.940819,2
3,0.103687,-1.319036,-1.352670,-1.364504,1.494486,1
4,-1.309502,1.750843,-0.557423,1.533576,-1.422048,2


## k-NN detector

### Detector with fixed params

In [10]:
detector1 = KNNFaultDetector(n_neighbors=3)
detector1.fit(X1, y1, optimize=False)
predictions = detector1.predict(X1)
print(detector1.score(X1, y1))
print(classification_report(y1, predictions))

0.55
              precision    recall  f1-score   support

           0       0.50      0.77      0.61        39
           1       0.58      0.34      0.43        32
           2       0.67      0.48      0.56        29

    accuracy                           0.55       100
   macro avg       0.58      0.53      0.53       100
weighted avg       0.57      0.55      0.54       100



In [11]:
knn_detector1 = KNNFaultDetector(
    n_neighbors=5,
    weights='distance',
    algorithm='auto',
    leaf_size=30,
    p=2
)

knn_detector1.fit(X2_train, y2_train, optimize=False)

predictions = knn_detector1.predict(X2_test)
print(f"Accuracy: {knn_detector1.score(X2_test, y2_test):.3f}")

Accuracy: 0.885


### k-NN randomized optimization

In [12]:
param_space = {
    'n_neighbors': [1, 20],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 50],
    'p': [1, 3],
    'metric': ['minkowski', 'euclidean', 'manhattan']
}

detector2 = KNNFaultDetector(param_search_space=param_space, n_iter=100, search_method='random')
detector2.fit(X1, y1, optimize=True)
print("Best params:", detector2.get_params())
predictions = detector2.predict(X1)
print(classification_report(y1, predictions))

Best params: {'algorithm': 'kd_tree', 'leaf_size': 31, 'metric': 'euclidean', 'n_neighbors': 14, 'p': 1, 'weights': 'uniform'}
              precision    recall  f1-score   support

           0       0.47      0.69      0.56        39
           1       0.38      0.28      0.32        32
           2       0.44      0.28      0.34        29

    accuracy                           0.44       100
   macro avg       0.43      0.42      0.41       100
weighted avg       0.43      0.44      0.42       100



### k-NN optuna optimization

In [13]:
detector3 = KNNFaultDetector(param_search_space=param_space, n_iter=20, search_method='optuna')
detector3.fit(X1, y1, optimize=True)
print("Best params:", detector3.get_params())
predictions = detector3.predict(X1)
print(classification_report(y1, predictions))

[I 2025-05-16 14:26:41,799] A new study created in memory with name: no-name-ac134ea7-8d63-4b2f-a267-ec698cb47cfd
[I 2025-05-16 14:26:41,807] Trial 0 finished with value: 0.2625 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'algorithm': 'ball_tree', 'leaf_size': 30, 'p': 2, 'metric': 'minkowski'}. Best is trial 0 with value: 0.2625.
[I 2025-05-16 14:26:41,825] Trial 1 finished with value: 0.375 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'kd_tree', 'leaf_size': 31, 'p': 2, 'metric': 'minkowski'}. Best is trial 1 with value: 0.375.
[I 2025-05-16 14:26:41,837] Trial 2 finished with value: 0.3625 and parameters: {'n_neighbors': 19, 'weights': 'distance', 'algorithm': 'brute', 'leaf_size': 48, 'p': 1, 'metric': 'manhattan'}. Best is trial 1 with value: 0.375.
[I 2025-05-16 14:26:41,847] Trial 3 finished with value: 0.4125 and parameters: {'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 19, 'p': 1, 'metric': 'minkowski'}

Best params: {'n_neighbors': 14, 'weights': 'uniform', 'algorithm': 'ball_tree', 'leaf_size': 19, 'p': 1, 'metric': 'minkowski'}
              precision    recall  f1-score   support

           0       0.45      0.69      0.55        39
           1       0.39      0.38      0.38        32
           2       0.56      0.17      0.26        29

    accuracy                           0.44       100
   macro avg       0.46      0.41      0.40       100
weighted avg       0.46      0.44      0.41       100



## Random forest detector

### Fixed params

In [14]:
rf_detector1 = RandomForestFaultDetector(
    n_estimators=50,
    max_depth=10,
    random_state=42
)

rf_detector1.fit(X1, y1, optimize=False)
predictions = rf_detector1.predict(X1)
print(classification_report(y1, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        39
           1       1.00      1.00      1.00        32
           2       1.00      1.00      1.00        29

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



### Randomized optimization

In [15]:
param_space = {
    'n_estimators': [50, 200],
    'max_depth': [5, 30],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

rf_detector2 = RandomForestFaultDetector(
    param_search_space=param_space,
    n_iter=20,
    search_method='random',
    random_state=42
)

rf_detector2.fit(X2_train, y2_train, optimize=True)

predictions = rf_detector2.predict(X2_test)
print(f"Accuracy: {rf_detector2.score(X2_test, y2_test):.3f}")

Accuracy: 0.880


### Optuna optimization

In [16]:
rf_detector3 = RandomForestFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='optuna',
    random_state=42
)
rf_detector3.fit(X2_train, y2_train, optimize=True)

predictions = rf_detector3.predict(X2_test)
print(f"Accuracy: {rf_detector3.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:26:46,657] A new study created in memory with name: no-name-05217626-1200-4ee1-a341-3ae99bd982dd
[I 2025-05-16 14:26:46,872] Trial 0 finished with value: 0.90625 and parameters: {'n_estimators': 106, 'max_depth': 29, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_features': 'sqrt', 'bootstrap': False, 'criterion': 'entropy'}. Best is trial 0 with value: 0.90625.
[I 2025-05-16 14:26:47,013] Trial 1 finished with value: 0.9125 and parameters: {'n_estimators': 53, 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini'}. Best is trial 1 with value: 0.9125.
[I 2025-05-16 14:26:47,266] Trial 2 finished with value: 0.915625 and parameters: {'n_estimators': 142, 'max_depth': 8, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False, 'criterion': 'gini'}. Best is trial 2 with value: 0.915625.
[I 2025-05-16 14:26:47,574] Trial 3 finished with value: 0.9015625 and para

Accuracy: 0.885


## SVM detector

### Fixed parameters

In [17]:
svm_detector1 = SVMFaultDetector(
    C=1.0,
    kernel='rbf',
    gamma='scale',
    probability=True,
    random_state=42
)
svm_detector1.fit(X1, y1, optimize=False)
predictions = svm_detector1.predict(X1)
print(classification_report(y1, predictions))

              precision    recall  f1-score   support

           0       0.63      1.00      0.77        39
           1       0.86      0.59      0.70        32
           2       0.94      0.52      0.67        29

    accuracy                           0.73       100
   macro avg       0.81      0.70      0.71       100
weighted avg       0.79      0.73      0.72       100



### Randomized optimization

In [18]:
param_space = {
    'C': [1e-3, 1e3],  
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [1e-3, 1e3],
    'degree': [2, 5],
    'coef0': [0.0, 1.0]
}

svm_detector2 = SVMFaultDetector(
    param_search_space=param_space,
    n_iter=20,
    search_method='random',
    random_state=42,
    probability=True
)

svm_detector2.fit(X2_train, y2_train, optimize=True)

predictions = svm_detector2.predict(X2_test)
print(f"Accuracy: {svm_detector2.score(X2_test, y2_test):.3f}")

Accuracy: 0.715


### Optuna optimization

In [19]:
svm_detector3 = SVMFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='optuna',
    random_state=42,
    probability=True
)
svm_detector3.fit(X2_train, y2_train, optimize=True)

predictions = svm_detector3.predict(X2_test)
print(f"Accuracy: {svm_detector3.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:26:52,173] A new study created in memory with name: no-name-e24f47de-6f82-4da4-9470-d5a626d440d8
[I 2025-05-16 14:26:52,189] Trial 0 finished with value: 0.8859375 and parameters: {'C': 0.1767016940294795, 'kernel': 'linear'}. Best is trial 0 with value: 0.8859375.
[I 2025-05-16 14:26:52,928] Trial 1 finished with value: 0.89375 and parameters: {'C': 0.008632008168602538, 'kernel': 'poly', 'gamma': 4.0428727350273315, 'degree': 4, 'coef0': 0.020584494295802447}. Best is trial 1 with value: 0.89375.
[I 2025-05-16 14:26:53,296] Trial 2 finished with value: 0.890625 and parameters: {'C': 659.871107205407, 'kernel': 'linear'}. Best is trial 1 with value: 0.89375.
[I 2025-05-16 14:26:53,322] Trial 3 finished with value: 0.5859375 and parameters: {'C': 0.012601639723276799, 'kernel': 'rbf', 'gamma': 0.055895242052179224}. Best is trial 1 with value: 0.89375.
[I 2025-05-16 14:26:54,045] Trial 4 finished with value: 0.8984375 and parameters: {'C': 4.689400963537685, 'kernel': 

Accuracy: 0.775


## Gradient boosting detector

### Fixed parameters

In [20]:
gb_detector1 = GradientBoostingFaultDetector(
    learning_rate=0.1,
    n_estimators=100,
    max_depth=5,
    random_state=42
)

gb_detector1.fit(X2_train, y2_train, optimize=False)

predictions = gb_detector1.predict(X2_test)
print(f"Accuracy: {gb_detector1.score(X2_test, y2_test):.3f}")

Accuracy: 0.890


### Randomized optimization

In [21]:
param_space = {
    'learning_rate': [0.01, 0.3],
    'n_estimators': [50, 200],
    'max_depth': [3, 10],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5],
    'subsample': [0.6, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

gb_detector2 = GradientBoostingFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='random',
    random_state=42
)

gb_detector2.fit(X2_train, y2_train, optimize=True)

predictions = gb_detector2.predict(X2_test)
print(f"Accuracy: {gb_detector2.score(X2_test, y2_test):.3f}")

Accuracy: 0.885


### Optuna optimization

In [22]:
gb_detector3 = GradientBoostingFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='optuna',
    random_state=42
)

gb_detector3.fit(X2_train, y2_train, optimize=True)

predictions = gb_detector3.predict(X2_test)
print(f"Accuracy: {gb_detector3.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:26:59,331] A new study created in memory with name: no-name-2fc3d4e8-fac6-4339-93ca-efdf6ce8dc6e
[I 2025-05-16 14:27:00,668] Trial 0 finished with value: 0.909375 and parameters: {'learning_rate': 0.03574712922600244, 'n_estimators': 193, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 1, 'subsample': 0.662397808134481, 'max_features': 'log2'}. Best is trial 0 with value: 0.909375.
[I 2025-05-16 14:27:01,051] Trial 1 finished with value: 0.903125 and parameters: {'learning_rate': 0.11114989443094977, 'n_estimators': 53, 'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 2, 'subsample': 0.6727299868828402, 'max_features': None}. Best is trial 0 with value: 0.909375.
[I 2025-05-16 14:27:01,642] Trial 2 finished with value: 0.9109375 and parameters: {'learning_rate': 0.04345454109729477, 'n_estimators': 93, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 2, 'subsample': 0.7465447373174767, 'max_features': 'log2'}. Best is trial 2 with val

Accuracy: 0.890


## LGBM detector

### Fixed parameters

In [23]:
lgbm_detector1 = LGBMFaultDetector(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=5,
    learning_rate=0.1,
    n_estimators=100,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    verbose=0
)


lgbm_detector1.fit(X2_train, y2_train, optimize=True)

predictions = lgbm_detector1.predict(X2_test)
print(f"Accuracy: {lgbm_detector1.score(X2_test, y2_test):.3f}")

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

### Randomized optimization

In [24]:
param_space = {
    'boosting_type': ['gbdt', 'dart'], 
    'n_estimators': [50, 200], 
    'num_leaves': [20, 100],           
    'learning_rate': [0.01, 0.3],     
    'max_depth': [3, 15],             
    'min_child_samples': [20, 20],        
    'subsample': [0.6, 1.0],         
    'colsample_bytree': [0.8, 0.8],        
    'reg_alpha': [0.001, 1],            
    'reg_lambda': [0.001, 1]             
}

lgbm_detector2 = LGBMFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='random',
    random_state=42,
    verbose=0
)

lgbm_detector2.fit(X2_train, y2_train, optimize=True)

predictions = lgbm_detector2.predict(X2_test)
print(f"Accuracy: {lgbm_detector2.score(X2_test, y2_test):.3f}")

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

### Optuna optimization

In [25]:
lgbm_detector2 = LGBMFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='optuna',
    random_state=42,
    verbose=0
)

lgbm_detector2.fit(X2_train, y2_train, optimize=True)

predictions = lgbm_detector2.predict(X2_test)
print(f"Accuracy: {lgbm_detector2.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:27:13,251] A new study created in memory with name: no-name-7f671dca-c906-47c1-b038-4922a95e8164
[I 2025-05-16 14:27:13,498] Trial 0 finished with value: 0.88125 and parameters: {'boosting_type': 'dart', 'num_leaves': 79, 'max_depth': 10, 'learning_rate': 0.01700037298921102, 'n_estimators': 73, 'min_child_samples': 20, 'subsample': 0.6232334448672797, 'colsample_bytree': 0.8, 'reg_alpha': 0.39676050770529875, 'reg_lambda': 0.06358358856676251}. Best is trial 0 with value: 0.88125.
[I 2025-05-16 14:27:13,866] Trial 1 finished with value: 0.8890625 and parameters: {'boosting_type': 'gbdt', 'num_leaves': 98, 'max_depth': 13, 'learning_rate': 0.020589728197687916, 'n_estimators': 77, 'min_child_samples': 20, 'subsample': 0.6733618039413735, 'colsample_bytree': 0.8, 'reg_alpha': 0.008179499475211672, 'reg_lambda': 0.03752055855124281}. Best is trial 1 with value: 0.8890625.
[I 2025-05-16 14:27:14,212] Trial 2 finished with value: 0.8921875 and parameters: {'boosting_type':

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

## XGB detector

### Fixed parameters

In [26]:
xgb_detector1 = XGBFaultDetector(
    max_depth=5,
    learning_rate=0.1,
    n_estimators=100,
    random_state=42
)

xgb_detector1.fit(X2_train, y2_train, optimize=False)

predictions = xgb_detector1.predict(X2_test)
print(f"Accuracy: {xgb_detector1.score(X2_test, y2_test):.3f}")

Accuracy: 0.870


c:\python\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:27:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


### Randomized optimization

In [27]:
param_space = {
    'max_depth': [3, 10],
    'learning_rate': [0.01, 0.3],
    'n_estimators': [50, 200],
    'gamma': [0.001, 0.5],
    'min_child_weight': [1, 10],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'reg_alpha': [0.001, 1],
    'reg_lambda': [0.001, 1]
}

xgb_detector2 = XGBFaultDetector(
    param_search_space=param_space,
    n_iter=25,
    search_method='random',
    random_state=42
)

xgb_detector2.fit(X2_train, y2_train, optimize=True)

predictions = xgb_detector2.predict(X2_test)
print(f"Accuracy: {xgb_detector2.score(X2_test, y2_test):.3f}")

c:\python\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Accuracy: 0.890


c:\python\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
42 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\python\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\python\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\python\Lib\site-packages\xgboost\sklearn.py", line 1512, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\python\Lib\site-pac

### Optuna optimization

In [28]:
xgb_detector3 = XGBFaultDetector(
    param_search_space=param_space,
    n_iter=25,
    search_method='optuna',
    random_state=42
)

xgb_detector3.fit(X2_train, y2_train, optimize=True)

predictions = xgb_detector3.predict(X2_test)
print(f"Accuracy: {xgb_detector3.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:27:39,872] A new study created in memory with name: no-name-e386fdc4-7eac-4a47-9aff-b8484b835fd3
[I 2025-05-16 14:27:40,073] Trial 0 finished with value: 0.90625 and parameters: {'max_depth': 5, 'learning_rate': 0.2536999076681772, 'n_estimators': 160, 'gamma': 0.29973058361432126, 'min_child_weight': 2, 'subsample': 0.662397808134481, 'colsample_bytree': 0.6232334448672797, 'reg_alpha': 0.39676050770529875, 'reg_lambda': 0.06358358856676251}. Best is trial 0 with value: 0.90625.
c:\python\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call las

Accuracy: 0.910


## Catboost detector

### Fixed parameters

In [29]:
cb_detector1 = CatBoostFaultDetector(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    random_state=42,
    verbose=True
)

cb_detector1.fit(X2_train, y2_train, optimize=False)

predictions = cb_detector1.predict(X2_test)
print(f"Accuracy: {cb_detector1.score(X2_test, y2_test):.3f}")

0:	learn: 1.0345191	total: 9.22ms	remaining: 4.6s
1:	learn: 0.9818152	total: 13.1ms	remaining: 3.26s
2:	learn: 0.9338495	total: 16.1ms	remaining: 2.67s
3:	learn: 0.8882445	total: 18.8ms	remaining: 2.33s
4:	learn: 0.8482475	total: 21.4ms	remaining: 2.12s
5:	learn: 0.8075747	total: 24.1ms	remaining: 1.99s
6:	learn: 0.7733112	total: 26.2ms	remaining: 1.84s
7:	learn: 0.7418368	total: 28.9ms	remaining: 1.77s
8:	learn: 0.7136008	total: 32.2ms	remaining: 1.76s
9:	learn: 0.6871510	total: 34.5ms	remaining: 1.69s
10:	learn: 0.6601624	total: 37.3ms	remaining: 1.66s
11:	learn: 0.6342170	total: 39.7ms	remaining: 1.61s
12:	learn: 0.6120377	total: 42.1ms	remaining: 1.58s
13:	learn: 0.5909336	total: 44.4ms	remaining: 1.54s
14:	learn: 0.5732914	total: 46.7ms	remaining: 1.51s
15:	learn: 0.5562961	total: 48.9ms	remaining: 1.48s
16:	learn: 0.5387725	total: 51.6ms	remaining: 1.46s
17:	learn: 0.5242903	total: 53.6ms	remaining: 1.44s
18:	learn: 0.5080745	total: 56.6ms	remaining: 1.43s
19:	learn: 0.4947426	to

### Randomized optimization

In [30]:
param_space = {
    'iterations': [100, 1000],
    'learning_rate': [0.01, 0.3],
    'depth': [4, 10],
    'l2_leaf_reg': [1, 10],
    'border_count': [32, 255],
    'random_strength': [0, 2],
    'bagging_temperature': [0, 1]
}

cb_detector2 = CatBoostFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='random',
    random_state=42,
    verbose=True
)

cb_detector2.fit(X2_train, y2_train, optimize=True)

predictions = cb_detector2.predict(X2_test)
print(f"Accuracy: {cb_detector2.score(X2_test, y2_test):.3f}")

Accuracy: 0.875


### Optuna optimization

In [31]:
cb_detector3 = CatBoostFaultDetector(
    param_search_space=param_space,
    n_iter=10,
    search_method='optuna',
    random_state=42,
    verbose=True
)

cb_detector3.fit(X2_train, y2_train, optimize=True)

predictions = cb_detector3.predict(X2_test)
print(f"Accuracy: {cb_detector3.score(X2_test, y2_test):.3f}")

[I 2025-05-16 14:32:10,540] A new study created in memory with name: no-name-3083e4f6-f3a5-4092-82ac-f7c6c88d9e8c
[I 2025-05-16 14:32:24,705] Trial 0 finished with value: 0.9046875 and parameters: {'iterations': 437, 'learning_rate': 0.2536999076681772, 'depth': 9, 'l2_leaf_reg': 3.968793330444372, 'border_count': 66, 'random_strength': 0.3119890406724053, 'bagging_temperature': 0.05808361216819946}. Best is trial 0 with value: 0.9046875.
[I 2025-05-16 14:33:23,170] Trial 1 finished with value: 0.90625 and parameters: {'iterations': 880, 'learning_rate': 0.07725378389307355, 'depth': 8, 'l2_leaf_reg': 1.0485387725194617, 'border_count': 249, 'random_strength': 1.6648852816008435, 'bagging_temperature': 0.21233911067827616}. Best is trial 1 with value: 0.90625.
[I 2025-05-16 14:33:25,039] Trial 2 finished with value: 0.9078125 and parameters: {'iterations': 263, 'learning_rate': 0.018659959624904916, 'depth': 6, 'l2_leaf_reg': 3.347776308515933, 'border_count': 128, 'random_strength': 0

0:	learn: 1.0640774	total: 834us	remaining: 690ms
1:	learn: 1.0297986	total: 1.83ms	remaining: 755ms
2:	learn: 0.9999637	total: 2.54ms	remaining: 697ms
3:	learn: 0.9693109	total: 3.37ms	remaining: 694ms
4:	learn: 0.9417328	total: 4.01ms	remaining: 660ms
5:	learn: 0.9151053	total: 4.62ms	remaining: 633ms
6:	learn: 0.8894779	total: 5.28ms	remaining: 619ms
7:	learn: 0.8654218	total: 6.03ms	remaining: 618ms
8:	learn: 0.8448227	total: 6.77ms	remaining: 616ms
9:	learn: 0.8237172	total: 7.48ms	remaining: 612ms
10:	learn: 0.8028535	total: 8.24ms	remaining: 612ms
11:	learn: 0.7831212	total: 8.94ms	remaining: 608ms
12:	learn: 0.7646503	total: 9.55ms	remaining: 598ms
13:	learn: 0.7470254	total: 10.2ms	remaining: 595ms
14:	learn: 0.7300906	total: 10.9ms	remaining: 591ms
15:	learn: 0.7141038	total: 11.8ms	remaining: 597ms
16:	learn: 0.6996334	total: 12.5ms	remaining: 594ms
17:	learn: 0.6846886	total: 13.1ms	remaining: 589ms
18:	learn: 0.6714638	total: 13.7ms	remaining: 585ms
19:	learn: 0.6579240	to